In [ ]:
pip install botorch

In [59]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

import pandas as pd

train_data = pd.read_excel('data_sample.xlsx')
#print(train_data)
train_X = train_data.iloc[:,[0,1]].values
#print(train_X)
train_X = torch.tensor(train_X, dtype=torch.float32)
#print(train_X)

train_Y = train_data.iloc[:,[2]].values
#print(train_Y)
train_Y = torch.tensor(train_Y, dtype=torch.float32)
#print(train_Y)

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition import qExpectedImprovement

UCB = UpperConfidenceBound(gp, beta=0.1)
qEI = qExpectedImprovement(gp, best_f=0.2)

from botorch.optim import optimize_acqf

#bounds = torch.stack([torch.zeros(2), torch.ones(2)])
#bounds = torch.tensor([[0.005, 0.5],[0.0406, 2]])
#bounds = torch.tensor([[0.005, 0.1],[0.0406, 3]])

bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(bounds)
candidate, acq_value = optimize_acqf(qEI, bounds=bounds, q=7, num_restarts=5, raw_samples=20)
print(candidate)
print(acq_value)

tensor([[5.0000e-02, 1.3926e+00],
        [5.0000e-02, 7.5281e-01],
        [1.0000e-03, 2.0647e-01],
        [5.0000e-02, 3.0000e+00],
        [5.0000e-02, 1.6209e+00],
        [5.0000e-02, 2.5215e+00],
        [5.0000e-02, 1.0000e-01]])
tensor(0.3526)


/usr/local/lib/python3.7/dist-packages/gpytorch/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  NumericalWarning,
